Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_s("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df);
first(df2, 5)

5×4 DataFrame
│ Row │ height   │ weight   │ age      │ male   │
│     │ Float64⍰ │ Float64⍰ │ Float64⍰ │ Int64⍰ │
├─────┼──────────┼──────────┼──────────┼────────┤
│ 1   │ 151.765  │ 47.8256  │ 63.0     │ 1      │
│ 2   │ 139.7    │ 36.4858  │ 63.0     │ 0      │
│ 3   │ 136.525  │ 31.8648  │ 65.0     │ 0      │
│ 4   │ 156.845  │ 53.0419  │ 41.0     │ 1      │
│ 5   │ 145.415  │ 41.2769  │ 51.0     │ 0      │

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchains);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height],
  "weight" => df2[:weight]);
# Sample using cmdstan
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME)
# Describe the draws
describe(chn)


File /Users/rob/.julia/dev/StanModels/scripts/04/tmp/weights.stan will be updated.

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
parameters        = alpha, beta, sigma

Empirical Posterior Estimates
──────────────────────────────────────────────────
parameters
        Mean     SD   Naive SE  MCSE   ESS
alpha 113.9010 1.9085   0.0302 0.0458 1000
 beta   0.9044 0.0421   0.0007 0.0010 1000
sigma   5.1004 0.1996   0.0032 0.0044 1000

Quantiles
──────────────────────────────────────────────────
parameters
        2.5%     25.0%    50.0%    75.0%    97.5% 
alpha 107.4730 112.6155 113.9535 115.1642 120.2550
 beta   0.7496   0.8765   0.9031   0.9326   1.0343
sigma   4.4695   4.9583   5.0989   5.2328   5.8714



Save the chains in a JLD file

In [7]:
serialize("m4.3s.jls", chn)

chn2 = deserialize("m4.3s.jls")

describe(chn2)

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
parameters        = alpha, beta, sigma

Empirical Posterior Estimates
──────────────────────────────────────────────────
parameters
        Mean     SD   Naive SE  MCSE   ESS
alpha 113.9010 1.9085   0.0302 0.0458 1000
 beta   0.9044 0.0421   0.0007 0.0010 1000
sigma   5.1004 0.1996   0.0032 0.0044 1000

Quantiles
──────────────────────────────────────────────────
parameters
        2.5%     25.0%    50.0%    75.0%    97.5% 
alpha 107.4730 112.6155 113.9535 115.1642 120.2550
 beta   0.7496   0.8765   0.9031   0.9326   1.0343
sigma   4.4695   4.9583   5.0989   5.2328   5.8714



Should be identical to earlier result

In [8]:
describe(chn2)

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
parameters        = alpha, beta, sigma

Empirical Posterior Estimates
──────────────────────────────────────────────────
parameters
        Mean     SD   Naive SE  MCSE   ESS
alpha 113.9010 1.9085   0.0302 0.0458 1000
 beta   0.9044 0.0421   0.0007 0.0010 1000
sigma   5.1004 0.1996   0.0032 0.0044 1000

Quantiles
──────────────────────────────────────────────────
parameters
        2.5%     25.0%    50.0%    75.0%    97.5% 
alpha 107.4730 112.6155 113.9535 115.1642 120.2550
 beta   0.7496   0.8765   0.9031   0.9326   1.0343
sigma   4.4695   4.9583   5.0989   5.2328   5.8714



End of `m4.3s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*